In [1]:
import geopandas
import pyproj
import pystac
import shapely
import stac2dcache

from stac2dcache.utils import get_asset

# Extract bounding box from Randolph Glacier Inventory (RGI)

We extract the bounding box in the projected (UTM) and lat/lon coordinates from the glacier ID. A buffer is applied to include some of the surrounding region in the area of interest. 

In [2]:
# input cell
MACAROON_PATH = "./macaroon.dat"

CATALOG_URL = "https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/eratosthenes/disk/red-glacier_copernicushub-gcp"

RGI_SHAPEFILE_URL = "https://webdav.grid.surfsara.nl:2880/pnfs/grid.sara.nl/data/eratosthenes/disk/GIS/Glacier/01_rgi60_Alaska.zip"
RGI_ID = "RGI60-01.19773"  # RGI ID of the Red Glacier
RGI_BUFFER = 5000  # buffer around glacier in meters

In [3]:
def _read_catalog(url, stac_io=None):
    """
    Read STAC catalog from URL

    :param url: urlpath to the catalog root
    :param stac_io (optional): STAC IO instance to read the catalog
    :return: PyStac Catalog object
    """
    url = url if url.endswith("catalog.json") else f"{url}/catalog.json"
    catalog = pystac.Catalog.from_file(url, stac_io=stac_io)
    return catalog


def _get_bbox_from_glacier(shapefile_url, id, crs, buffer, filesystem):
    """
    Compute bounding box for a glacier from the Randolph Glacier Inventory

    :param shapefile_url: urlpath to the RGI shapefile
    :param id: RGI glacier ID
    :param crs: output CRS
    :param buffer: buffer region, in the output CRS units
    :param filesystem: filesystem object
    :return: bounding box
    """
    with filesystem.open(shapefile_url) as f:
        glaciers = geopandas.read_file(f)

    glacier = glaciers[glaciers["RGIId"] == id]
    glacier_geom = glacier.geometry.item()

    # transform to new CRS
    utm = pyproj.CRS(crs)
    wgs84 = pyproj.CRS("EPSG:4326")
    transformer = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True)

    glacier_geom_utm = shapely.ops.transform(
        transformer.transform, glacier_geom
    )
    glacier_geom_utm = glacier_geom_utm.buffer(buffer).envelope
    return glacier_geom_utm.bounds


def _get_polygon_wgs84(bbox, crs):
    """
    Transform bbox in a given CRS to a WGS84 Polygon
    
    :param bbox: bounding box
    :param crs: coordinate reference system
    :return: converted polygon
    """
    utm = pyproj.CRS(crs)
    wgs84 = pyproj.CRS("EPSG:4326")
    transformer = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True)
    
    polygon_utm = shapely.geometry.Polygon.from_bounds(*bbox)
    return shapely.ops.transform(
        transformer.transform, polygon_utm
    )

In [4]:
# configure connection to dCache
stac2dcache.configure(token_filename=MACAROON_PATH)

In [5]:
# get CRS from one of the catalog items
catalog = _read_catalog(CATALOG_URL, stac_io=stac2dcache.stac_io)
item = next(catalog.get_all_items())  # get a random item
_band = get_asset(
    catalog,
    asset_key="B02",
    item_id=item.id,
    filesystem=stac2dcache.fs,
    load=False
)
crs = _band.rio.crs

GDAL headers saved to: /tmp/tmpyyzd9ltv


In [6]:
# compute bounding box
bbox = _get_bbox_from_glacier(
    RGI_SHAPEFILE_URL,
    RGI_ID,
    _band.rio.crs,
    RGI_BUFFER,
    stac2dcache.fs
)

# convert back to WGS84
polygon_wgs84 = _get_polygon_wgs84(bbox, _band.rio.crs)

In [7]:
print("Bounding box in UTM (rounded to integer): ", [round(el) for el in bbox])
print("Polygon in WGS84: ", polygon_wgs84)

Bounding box in UTM (rounded to integer):  [490229, 6642656, 516134, 6660489]
Polygon in WGS84:  POLYGON ((-153.17475999752926 59.921268739678986, -153.1756065979441 60.08139181090761, -152.71003577814346 60.08119061686564, -152.71143368231296 59.92106883348645, -153.17475999752926 59.921268739678986))
